In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds

def foo(x):
    return sum( (x-np.array([1,2,3]))**2)

In [2]:
x0 = np.array([1,4,2])
print(foo(x0))

bounds = Bounds([-100,-100,-100],[100,100,100])
res = minimize(foo,x0, method='nelder-mead', bounds=bounds, options={'verbose':0})
print(res.x)

5
[0.99995732 1.9999766  3.00002708]


C:\Users\evert\AppData\Local\Temp\ipykernel_20576\3293204988.py:5: OptimizeWarning: Unknown solver options: verbose
  res = minimize(foo,x0, method='nelder-mead', bounds=bounds, options={'verbose':0})


# Recover the rotation angles

In [3]:
import cv2
# True params:
angles_true = np.array([0,50,30])
R_true,_ = cv2.Rodrigues(np.deg2rad(angles_true))
T_true = np.array([0,1,3]).reshape(3,1)

Pw = np.array([[0,0,0],[0,1,1],[0,4,1],[0,5,2],[0,5,5],[4,3,-1]]).T
Pl = R_true.dot(Pw) + T_true

In [11]:
# with this data: reestimate the angles:
def error_angles(ang_and_T, Pw, Pl):
    angles_hat = np.array(ang_and_T[0:3])
    T_hat = np.array(ang_and_T[3:])
    R_hat,_ = cv2.Rodrigues(np.deg2rad(angles_hat))

    Pl_hat = R_hat.dot(Pw) + T_hat.reshape(3,1)
    error = np.mean(np.sum((Pl - Pl_hat)**2,axis=0,keepdims=True), axis=1, keepdims=True)
    return error

ang_and_T = [0,0,31,0,1,3]
err = error_angles(ang_and_T,Pw,Pl)
print(err)



[[5.21090317]]


In [5]:
from scipy.optimize import minimize
from scipy.optimize import Bounds

# find the angles and T using optimization:
x0 = np.array([10,20,40,10.1,20.5,3.5])
x0mins = [-90,-90,-90,-0.004,-100,-100]
x0max =  [ 90, 90, 90, 100, 100, 100]
bounds = Bounds(x0mins, x0max)

res = minimize(error_angles,x0,args=(Pw,Pl),method='nelder-mead',options={'display':True})
print(res.x)

C:\Users\evert\AppData\Local\Temp\ipykernel_20576\2597574607.py:10: OptimizeWarning: Unknown solver options: display
  res = minimize(error_angles,x0,args=(Pw,Pl),method='nelder-mead',options={'display':True})


[-2.57239528e-06  5.00000086e+01  2.99999916e+01  1.56235703e-06
  9.99999993e-01  3.00000150e+00]


# Optimization problem with worl points and unitary directions

In [18]:
import cv2
# True params:
angles_true = np.array([0,50,30])
R_true,_ = cv2.Rodrigues(np.deg2rad(angles_true))
T_true = np.array([0,1,3]).reshape(3,1)

Pw = np.array([[0,0,0],[0,1,1],[0,4,1],[0,5,2],[0,5,5],[4,3,-1]]).T
Pl = R_true.dot(Pw) + T_true
Ul = Pl/np.sqrt(np.sum(Pl**2,axis=0,keepdims=True))

print(Ul.shape)
print(np.sum(Ul**2,0))

(3, 6)
[1. 1. 1. 1. 1. 1.]


In [19]:
# with this data: reestimate the angles:
def error_angles_with_Ul(ang_and_T, Pw, Ul):
    angles_hat = np.array(ang_and_T[0:3])
    T_hat = np.array(ang_and_T[3:])
    R_hat,_ = cv2.Rodrigues(np.deg2rad(angles_hat))

    Pl_hat = R_hat.dot(Pw) + T_hat.reshape(3,1)
    Ul_hat = Pl_hat/np.sqrt(np.sum(Pl_hat**2,axis=0,keepdims=True))

    error = np.mean(np.sum((Ul - Ul_hat)**2,axis=0,keepdims=True), axis=1, keepdims=True)
    return error

ang_and_T = [0,0,31,0,1,3]
err = error_angles_with_Ul(ang_and_T,Pw,Ul)
print(err)


[[0.08390432]]


In [20]:
from scipy.optimize import minimize
from scipy.optimize import Bounds

# find the angles and T using optimization:
x0 = np.array([10,20,40,10.1,20.5,3.5])
x0mins = [-90,-90,-90,-0.004,-100,-100]
x0max =  [ 90, 90, 90, 100, 100, 100]
bounds = Bounds(x0mins, x0max)

res = minimize(error_angles_with_Ul,x0,args=(Pw,Ul),method='nelder-mead',options={'display':True})
print(angles_true, T_true)
print(res.x)

[ 0 50 30] [[0]
 [1]
 [3]]
[ 1.56892494e-04  4.99998625e+01  2.99997780e+01 -2.26588348e-06
  9.99992515e-01  2.99998586e+00]


C:\Users\evert\AppData\Local\Temp\ipykernel_20576\415718137.py:10: OptimizeWarning: Unknown solver options: display
  res = minimize(error_angles_with_Ul,x0,args=(Pw,Ul),method='nelder-mead',options={'display':True})
